## QGIS in Python

To run QGIS Python code, you need to have QGIS and its Python bindings installed on your system. Here's a general guide on how to do it:

Install QGIS: The installation process depends on your operating system. You can download QGIS from the official website (https://www.qgis.org/en/site/forusers/download.html). Follow the instructions provided for your specific operating system.

Install Python bindings for QGIS (PyQGIS): PyQGIS is included **by default** with the QGIS installation. You just need to make sure that you're using the Python version that comes with QGIS, or that your Python environment has access to the QGIS modules.

For Windows, the Python that comes with QGIS is usually located in the bin folder of the QGIS installation directory (e.g., C:\Program Files\QGIS 3.16\bin). You can add this path to your system's PATH environment variable.

For Linux, QGIS and its Python bindings can be installed using the package manager of your distribution. For example, on Ubuntu, you can use the following commands:

                sudo apt-get update
                sudo apt-get install qgis python-qgis

Install additional Python packages: If your script uses additional Python packages (like os or QImage from PyQt5), you need to install them as well. This can usually be done with pip, Python's package installer. For example: 

            pip install PyQt5

Remember to use the correct pip associated with your QGIS Python installation.

### Check if QGIS and its Python bindings (PyQGIS) are installed correctly.

In [ ]:
from qgis.core import QgsApplication

try:
    QgsApplication
    print("QGIS and PyQGIS are installed correctly.")
except ImportError:
    print("QGIS or PyQGIS is not installed correctly.")

### False Color Composite (FCC)

For a false color composite in Landsat 8, you can use bands 5, 4, and 3 to create an image with near-infrared, red, and green components, respectively. This combination is often referred to as the "NIR-Red-Green" composite and is useful for vegetation analysis and land cover classification.

In [ ]:
import os
from qgis.core import QgsApplication, QgsVectorLayer, QgsProject, QgsRasterLayerSettings, QgsSingleBandPseudoColorRenderer
from qgis.PyQt.QtGui import QImage, QColor
from qgis.PyQt.QtGui import QPainter

# Define paths
PROJECT_PATH = "/path/to/your/project.qgs"
RASTER_PATHS = ["/path/to/red_band.tif", "/path/to/green_band.tif", "/path/to/blue_band.tif"]
IMAGE_PATH = "/path/to/output/false_color_composite.jpg"

def start_qgis():
    """
    Start the QGIS application.

    Returns:
        QgsApplication: The QGIS application instance.
    """
    qgs = QgsApplication([], False)
    qgs.initQgis()
    return qgs

def load_layers():
    """
    Load raster layers into the QGIS project.

    Returns:
        QgsProject: The QGIS project instance.
        QgsRasterLayer: The loaded raster layer.
    Raises:
        FileNotFoundError: If the project file or any of the raster files are not found.
    """
    if not os.path.exists(PROJECT_PATH):
        raise FileNotFoundError(f"Project file {PROJECT_PATH} not found.")
    project = QgsProject.instance()
    project.read(PROJECT_PATH)
    for raster_path in RASTER_PATHS:
        if not os.path.exists(raster_path):
            raise FileNotFoundError(f"Raster file {raster_path} not found.")
        layer = QgsRasterLayer(raster_path, "")
        project.addMapLayer(layer)
    return project, layer

def create_false_color_composite(layer, red_band=1, green_band=2, blue_band=3):
    """
    Create a false color composite from the given raster layer.

    Args:
        layer (QgsRasterLayer): The raster layer to create the composite from.
        red_band (int): The band index for the red channel (default: 1).
        green_band (int): The band index for the green channel (default: 2).
        blue_band (int): The band index for the blue channel (default: 3).
    """
    settings = QgsRasterLayerSettings()
    settings.renderer = QgsSingleBandPseudoColorRenderer(layer.dataProvider(), red_band, QgsSingleBandPseudoColorRenderer.createDefaultColorTable(3))
    layer.setRendererSettings(settings)

def refresh_and_export(iface):
    """
    Refresh the map canvas and export the image.

    Args:
        iface: The QGIS interface reference in Jupyter notebook.
    """
    iface.mapCanvas().refreshAllLayers()
    image = QImage(iface.mapCanvas().size(), QImage.Format_ARGB32)
    image.fill(QColor(255, 255, 255).rgb())
    painter = QPainter(image)
    iface.mapCanvas().render(painter)
    image.save(IMAGE_PATH)

def main():
    """
    Main function to execute the workflow.
    """
    qgs = start_qgis()
    project, layer = load_layers()
    create_false_color_composite(layer)
    iface = iface # Change this to your QGIS interface reference in Jupyter notebook
    refresh_and_export(iface)
    project.write()
    qgs.exitQgis()

if __name__ == "__main__":
    main()